In [1]:
!pip install -q pyyaml h5py

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import os

import tensorflow as tf
from tensorflow import keras

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

print(tf.__version__)

2.3.0


In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784, )),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    
    model.compile(optimizer='adam',
                 loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    
    return model

model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

model.fit(train_images,
         train_labels,
         epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

Epoch 1/10
27/32 [========================>.....] - ETA: 0s - loss: 1.2338 - accuracy: 0.6447
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 1.1700 - accuracy: 0.6650 - val_loss: 0.7304 - val_accuracy: 0.7780
Epoch 2/10
 1/32 [..............................] - ETA: 0s - loss: 0.4065 - accuracy: 0.9062
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.4171 - accuracy: 0.8820 - val_loss: 0.5064 - val_accuracy: 0.8480
Epoch 3/10
 1/32 [..............................] - ETA: 0s - loss: 0.1937 - accuracy: 0.9688
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.2766 - accuracy: 0.9310 - val_loss: 0.5266 - val_accuracy: 0.8310
Epoch 4/10
 1/32 [..............................] - ETA: 0s - loss: 0.3012 - accuracy: 0.9062
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 3

In [7]:
ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [20]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3564 - accuracy: 0.0640
훈련되지 않은 모델의 정확도:  6.40%


In [21]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)

32/32 - 0s - loss: 0.3869 - accuracy: 0.8730


In [31]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images, train_labels, epochs=50, callbacks=[cp_callback],
         validation_data=(test_images, test_labels), verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [32]:
ls {checkpoint_dir}

checkpoint                        cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index                cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index                cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index                cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index                cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index                cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [33]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [34]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4873 - accuracy: 0.8780
복원된 모델의 정확도: 87.80%


In [35]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('./checkpoints/my_checkpoint')

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4873 - accuracy: 0.8780
복원된 모델의 정확도: 87.80%


model.save 메서드를 호출하여 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장합니다.

In [36]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.1880 - accuracy: 0.6600
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4400 - accuracy: 0.8820
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2806 - accuracy: 0.9330
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2066 - accuracy: 0.9440
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1496 - accuracy: 0.9660
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [37]:
!ls saved_model/

my_model


In [38]:
!ls saved_model/my_model/

assets	saved_model.pb	variables


In [44]:
new_model = tf.keras.models.load_model('saved_model/my_model/')

new_model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_25 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_51 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [45]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4253 - accuracy: 0.0820
복원된 모델의 정확도:  8.20%
(1000, 10)


In [52]:
model = create_model()
model.fit(train_images, train_labels, epochs=10)

model.save('my_model.h5')

Epoch 1/10
32/32 [==============================] - 0s 1ms/step - loss: 1.1420 - accuracy: 0.6810
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4092 - accuracy: 0.8880
Epoch 3/10
32/32 [==============================] - 0s 1ms/step - loss: 0.2720 - accuracy: 0.9330
Epoch 4/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1997 - accuracy: 0.9520
Epoch 5/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1474 - accuracy: 0.9680
Epoch 6/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1234 - accuracy: 0.9730
Epoch 7/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0822 - accuracy: 0.9890
Epoch 8/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0659 - accuracy: 0.9900
Epoch 9/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0473 - accuracy: 0.9990
Epoch 10/10
32/32 [==============================] - 0s 1ms/step - loss: 0.0361 - accuracy: 1.0000


In [53]:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_28 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [54]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4100 - accuracy: 0.0900
복원된 모델의 정확도:  9.00%
